In [4]:
import numpy as np 
import pandas as pd 
import netCDF4
import pickle 
import os
import pandas as pd
import h5py 
import numpy as np

def Reshape( X ):
    x =[ ]
    for i in range(X.shape[1]):
        x.append(X[: , i  , :])
    return np.array(x)

def rescale( data  ,Ref ) :
    means = np.mean(Ref , axis = 0 )
    std = np.std( Ref , axis =0)
    return (data -means ) / std

def Dict_to_hdf(Dict , of ):
    HDF = h5py.File(of, 'w')
    for k in Dict :    
        test = HDF.create_dataset( k , data = np.array(Dict[k] ))
    HDF.close()   


def nc_to_pd(nc):
    try:
        Keys = list(nc.keys())
    except:
        Keys = list(nc.variables)
    Panda ={ }
    for k in Keys:
        Panda[k]= np.array(nc[k] )    
    return pd.DataFrame(Panda)

def DF_bites_to_str(DF):
        
    for k in DF :
        if type(DF[k][0])  == np.bytes_ :
            DF[k]  = np.array( DF[k]).astype(str)
    return DF


class Dataframe_multi_dim :

    def __init__(self  , Dict ):
        self.df  = Dict
        self.Keys = list(self.df.keys())
        
    def Querry(  self , Key  ,  condition  , value) : 
        
        self.df_k = pd.DataFrame(self.df[Key] , columns= [Key])

        if condition  == '>=' : 
            querry = self.df_k[self.df_k[Key]  >= value ] 
        if condition  == '>' : 
            querry = self.df_k[self.df_k[Key]  > value ] 
        if condition  == '==' : 
            querry = self.df_k[self.df_k[Key]  == value ] 
        if condition  == '<' : 
            querry = self.df_k[self.df_k[Key]  < value ] 
        if condition  == '<=' :
            querry = self.df_k[self.df_k[Key]  <= value ] 
        if condition  == '!=' :
            querry = self.df_k[self.df_k[Key]  != value ] 
        self.Index = np.array(querry.index)
        self.out_df  ={ }
        
        for k in self.Keys  :
            self.out_df[k]  = np.array(self.df[k])[self.Index]
        self.df  = self.out_df
        self.Keys = list(self.df.keys())
        return Dataframe_multi_dim(self.df )





In [45]:

Folder = "D:/Projects/precipitation_AUS/dataset/0025/"
files = os.listdir(Folder)
hdfs = []

for file in files :
    hdf = h5py.File(Folder + file , 'r+')
    hdfs.append( hdf)
    print(hdf.keys())
    
needed_keys = [ 'NAME'  , 'LATITUDE' , 'LONGITUDE' ,  'YEAR'  , 'MONTH'   ,  'DAY'  , 'PRCP'  , 'STATE_CODE' , 'PDIR'  , 'ERA5_tp'  , 
'ERA5_cape' , 'ERA5_tcw'   , 'CAL_P' ,'tp_7'  ,'t2m_7' ] 

DF = {}
for k in needed_keys :
    for hdf in hdfs :
        if (k not in list(DF.keys())) and (k in list(hdf.keys())):
            DF[k] = np.array(hdf[k])
            

<KeysViewHDF5 ['CAL_P', 'CAL_P(-1)', 'CAL_P(-2)', 'CAL_P(-3)', 'CAL_P(-4)', 'CAL_P(-5)', 'CAL_P_ave', 'DAY', 'DEWP', 'DEWP_ATTRIBUTES', 'Date', 'ELEVATION', 'ERA5_cape', 'ERA5_cape(-1)', 'ERA5_cape(-2)', 'ERA5_cape(-3)', 'ERA5_cape(-4)', 'ERA5_cape(-5)', 'ERA5_msl', 'ERA5_msl(-1)', 'ERA5_msl(-2)', 'ERA5_msl(-3)', 'ERA5_msl(-4)', 'ERA5_msl(-5)', 'ERA5_t2m', 'ERA5_t2m(-1)', 'ERA5_t2m(-2)', 'ERA5_t2m(-3)', 'ERA5_t2m(-4)', 'ERA5_t2m(-5)', 'ERA5_tcw', 'ERA5_tcw(-1)', 'ERA5_tcw(-2)', 'ERA5_tcw(-3)', 'ERA5_tcw(-4)', 'ERA5_tcw(-5)', 'ERA5_tp', 'ERA5_tp(-1)', 'ERA5_tp(-2)', 'ERA5_tp(-3)', 'ERA5_tp(-4)', 'ERA5_tp(-5)', 'FRSHTT', 'GUST', 'HQ_P', 'HQ_P(-1)', 'HQ_P(-2)', 'HQ_P(-3)', 'HQ_P(-4)', 'HQ_P(-5)', 'LATITUDE', 'LONGITUDE', 'LSTM_RA_0', 'LSTM_RA_0(-1)', 'LSTM_RA_0(-2)', 'LSTM_RA_0(-3)', 'LSTM_RA_0(-4)', 'LSTM_RA_1', 'LSTM_RA_2', 'LSTM_RA_3', 'LSTM_RA_3(-1)', 'LSTM_RA_3(-2)', 'LSTM_RA_3(-3)', 'LSTM_RA_3(-4)', 'LSTM_RA_4', 'LSTM_RA_4(-4)', 'MAX', 'MAX_ATTRIBUTES', 'MIN', 'MIN_ATTRIBUTES', 'MON

In [54]:
ord_keys =  [ 'tp_7'  , 't2m_7']
obs_key = 'PRCP'

In [55]:
MDF =  Dataframe_multi_dim(DF)
train =  MDF.Querry('YEAR'  ,  '<='  , 2020 )
test =  MDF.Querry('YEAR'  ,  '>'  , 2021 )

In [56]:

def Read(Dict  , ord_keys  , obs_key) : 
    x_train = [ ]
    obs_train  = Dict[obs_key]  
    for i in range(len(obs_train)) :
        In_3d  = [ ]
        for k in ord_keys :
            In_3d.append(Dict[k][i])
        x_train.append( In_3d)
    return np.array(x_train) ,  np.array(obs_train)  


In [57]:
x_train , obs_train  =  Read(train.df, ord_keys  , obs_key)
x_test  , obs_test   = Read(test.df, ord_keys  , obs_key)


In [58]:
x_train.shape

(1013846, 2, 7, 7)